# Load and preprocess data

In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
from torchsummary import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
batch_size = 256
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # <-- Chuẩn ImageNet
    transforms.RandomErasing(p=0.75, scale=(0.01, 0.3), ratio=(1.0, 1.0), value=0, inplace=True),
])
train_set = CIFAR10(
    root = './data',
    train = True,
    download = True,
    transform= transform
)
val_set = CIFAR10(
    root = './data',
    train = False,
    download = True,
    transform= transform
)
train_loader = DataLoader(
    train_set,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 6
)
test_loader = DataLoader(
    val_set,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 6
)

# Model Construction
## **Mô tả các khối của mô hình `CIFAR10_CNN`**
Mô hình CNN này được thiết kế để phân loại ảnh từ tập dữ liệu CIFAR-10 (ảnh kích thước 32×32 với 3 kênh màu). Mô hình gồm hai phần chính: **Feature Extractor** (trích xuất đặc trưng) và **Classifier** (phân loại).

---

## **1️⃣ Feature Extractor (Trích xuất đặc trưng)**
📌 **Mục tiêu:** Trích xuất thông tin quan trọng từ ảnh bằng các lớp tích chập (`Conv2d`), chuẩn hóa (`BatchNorm2d`), và phi tuyến (`ReLU`), đồng thời giảm kích thước qua các lớp `MaxPool2d`.

| **Layer**        | **Chi tiết** |
|-----------------|-------------|
| **Conv2d(3→64)** | Tích chập 3x3, padding 1 (giữ nguyên kích thước), tạo 64 kênh đặc trưng |
| **BatchNorm2d(64)** | Chuẩn hóa theo batch giúp ổn định quá trình huấn luyện |
| **ReLU** | Kích hoạt phi tuyến |
| **Conv2d(64→64)** | Tích chập 3x3, padding 1 |
| **BatchNorm2d(64)** | Chuẩn hóa |
| **ReLU** | Kích hoạt |
| **MaxPool2d(2x2, stride=2)** | Giảm kích thước còn 16×16 |
| **Conv2d(64→128)** | Tích chập 3x3, tạo 128 kênh |
| **BatchNorm2d(128)** | Chuẩn hóa |
| **ReLU** | Kích hoạt |
| **Conv2d(128→128)** | Tích chập 3x3 |
| **BatchNorm2d(128)** | Chuẩn hóa |
| **ReLU** | Kích hoạt |
| **MaxPool2d(2x2, stride=2)** | Giảm kích thước còn 8×8 |
| **Conv2d(128→256)** | Tích chập 3x3, tạo 256 kênh |
| **BatchNorm2d(256)** | Chuẩn hóa |
| **ReLU** | Kích hoạt |
| **Conv2d(256→256)** | Tích chập 3x3 |
| **BatchNorm2d(256)** | Chuẩn hóa |
| **ReLU** | Kích hoạt |
| **MaxPool2d(2x2, stride=2)** | Giảm kích thước còn 4×4 |

---

## **2️⃣ Classifier (Phân loại)**
📌 **Mục tiêu:** Biến đổi đặc trưng đã trích xuất thành một dự đoán phân loại với 10 lớp.

| **Layer**        | **Chi tiết** |
|-----------------|-------------|
| **AdaptiveAvgPool2d(1x1)** | Giảm kích thước xuống 1×1 |
| **Flatten()** | Biến tensor thành vector 1D |
| **Linear(256→128)** | Fully connected layer với 128 neuron |
| **ReLU** | Kích hoạt phi tuyến |
| **Dropout(0.5)** | Loại bỏ ngẫu nhiên 50% neuron để tránh overfitting |
| **Linear(128→10)** | Fully connected layer để phân loại 10 lớp |

---

## **3️⃣ Khởi tạo trọng số**
📌 **Dùng He Initialization (Kaiming Normal) để khởi tạo trọng số giúp ổn định quá trình huấn luyện**

- `Conv2d`, `Linear`: `kaiming_normal_`
- `BatchNorm2d`: Hệ số scale = 1, bias = 0

```python
for m in model.modules():
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)
```

---

## **4️⃣ Tổng kết**
- Mô hình có **3 khối convolutional**, mỗi khối gồm 2 lớp tích chập + BatchNorm + ReLU.
- **MaxPooling** được sử dụng sau mỗi khối để giảm kích thước feature map.
- **AdaptiveAvgPool2d(1x1)** giúp chuyển feature map thành vector 1D.
- **Dropout(0.5)** giúp tránh overfitting.
- **Khởi tạo trọng số** bằng He Initialization giúp tăng tốc độ hội tụ.


In [ ]:
class CIFAR10_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10_CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Downscale to 1x1
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

        # 🔥 Khởi tạo trọng số bằng He Initialization
        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
model = CIFAR10_CNN()
model.to(device)
summary(model, input_size=(3, 32, 32))

In [ ]:
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    running_correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
    accuracy = 100 * running_correct/total
    test_loss = test_loss/len(test_loader)
    return test_loss, accuracy

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 1e-3)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

In [ ]:
def train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, max_epoch=50):
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(max_epoch):
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0

        for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epoch}", unit="batch")):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()

        epoch_accuracy = 100 * running_correct / total
        epoch_loss = running_loss / (i+1)

        # Đánh giá trên tập kiểm tra
        test_loss, test_accuracy = evaluate(model, test_loader, criterion)

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        # Cập nhật learning rate bằng scheduler
        lr_scheduler.step(test_loss)  # Dùng test_loss làm input cho scheduler

        # In kết quả sau mỗi epoch
        print(f"Epoch [{epoch+1}/{max_epoch}], "
              f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, "
              f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    return train_losses, train_accuracies, test_losses, test_accuracies

In [ ]:
train_losses, train_accuracies, test_losses, test_accuracies = train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, max_epoch=50)

In [ ]:
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# Vẽ loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss", marker="o")
plt.plot(epochs, test_losses, label="Test Loss", marker="o")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid()

# Vẽ accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label="Train Accuracy", marker="o")
plt.plot(epochs, test_accuracies, label="Test Accuracy", marker="o")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training & Validation Accuracy")
plt.legend()
plt.grid()

plt.show()

In [ ]:
save_path = './model.pth'
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")